In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path

In [30]:
tokenizer = ByteLevelBPETokenizer(lowercase=False)

In [17]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-academia.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-wiki.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-iium.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-news.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-parliament.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-watpadd.txt
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/dataset/bahasa-asr-train.json
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/dataset/bahasa-asr-test.json

In [25]:
import json

with open('bahasa-asr-train.json') as fopen:
    data = json.load(fopen)['Y']
    
with open('bahasa-asr-test.json') as fopen:
    data.extend(json.load(fopen)['Y'])
    
data = list(filter(None, data))
data[:10]

['inilah dunia kecil',
 'itu tadi menarik ya',
 'pelayan restoran ini kelihatannya sibuk',
 'apakah anda pernah makan serangga',
 'aku pernah ingin menjadi seorang astronom fisika',
 'saya sedang mencari dompet hitam seperti ini',
 'apakah anda percaya pada tuhan',
 'semuanya sedang menunggu guru di kelas',
 'anak-anak kecil suka dengan buku tentang dinosaurus dan monster',
 'ini yang dilakukan mary untuk hidup']

In [26]:
with open('asr-text.txt', 'w') as fopen:
    fopen.write('\n'.join(data))

In [27]:
files = ['asr-text.txt', 'filtered-dumping-academia.txt', 'filtered-dumping-wiki.txt',
        'dumping-iium.txt', 'dumping-news.txt', 'dumping-parliament.txt']

In [28]:
!cat {' '.join(files)} >> train-v3.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
!wc -w train-v3.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
183877906 train-v3.txt


In [31]:
tokenizer.train(files=files, vocab_size=32000, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])

In [33]:
!mkdir roberta_bpe

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: cannot create directory ‘roberta_bpe’: File exists


In [34]:
tokenizer.save_model('roberta_bpe')

['roberta_bpe/vocab.json', 'roberta_bpe/merges.txt']

In [35]:
import torch
import numpy as np
import transformers
from transformers import RobertaTokenizer, AutoModelForMaskedLM, RobertaConfig, RobertaForMaskedLM

In [36]:
tokenizer = RobertaTokenizer.from_pretrained('./roberta_bpe', do_lower_case = False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [37]:
tokenizer.save_pretrained('malay-cased-roberta-base')

('malay-cased-roberta-base/tokenizer_config.json',
 'malay-cased-roberta-base/special_tokens_map.json',
 'malay-cased-roberta-base/vocab.json',
 'malay-cased-roberta-base/merges.txt',
 'malay-cased-roberta-base/added_tokens.json')

In [38]:
tokenizer

PreTrainedTokenizer(name_or_path='./roberta_bpe', vocab_size=32000, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})